# NEIGHBORING TORONTO CITY

## EVERYTHING IS IN ONE NOTEBOOK ITSELF

#### IMPORT LIBRARIES

In [100]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### GET THE DATA OF TORONTO CITY INTO A DATAFRAME

In [101]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)[0]
data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### CLEANING UP THE DATA

In [102]:
Toronto_data = data.where(data['Borough'] != 'Not assigned').dropna()
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [103]:
Toronto_data.sort_values('Postal Code', ascending = True, inplace = True)
Toronto_data = Toronto_data.reset_index().drop('index', axis = 1)
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [104]:
for index, row in Toronto_data.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] == row['Borough']
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [105]:
Toronto_data.shape

(103, 3)

#### TRIED FINDING THE LATITUDE AND LONGITUDE OF THE POSTAL CODE (USING GEOPY)

In [82]:
# Create new columns
Toronto_data['Latitude'] = None
Toronto_data['Longitude'] = None
geolocator = Nominatim(user_agent="to_explorer")

# iterate over every row to find its latitude and logitude
for index, row in Toronto_data.iterrows():
    Neighbourhood = row['Neighbourhood'].split(',')[0]
    print(Neighbourhood)
    # Geopy does not provide coordinates for postal codes and od few neighbourhoods
    if Neighbourhood == 'Caledonia-Fairbanks' or Neighbourhood == 'Del Ray':
        continue
    address = '{}, Toronto, Ontario'.format(Neighbourhood)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    # Add it to the new columns
    row['Latitude'] = latitude
    row['Longitude'] = longitude
    
    


Parkwoods
Victoria Village
Regent Park
Lawrence Manor
Queen's Park
Islington Avenue
Malvern
Don Mills
Parkview Hill
Garden District
Glencairn
West Deane Park
Rouge Hill
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
Guildwood
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Thorncliffe Park
Richmond
Dufferin
Scarborough Village
Fairview
Northwood Park
East Toronto
Harbourfront East
Little Portugal
Kennedy Park
Bayview Village
Downsview
The Danforth West
Toronto Dominion Centre
Brockton
Golden Mile
York Mills
Downsview
India Bazaar
Commerce Court
North Park
Humber Summit
Cliffside
Willowdale
Downsview
Studio District
Bedford Park
Del Ray
Humberlea
Birch Cliff
Willowdale
Downsview
Lawrence Park
Roselawn
Runnymede
Weston
Dorset Park
York Mills West
Davisville North
Forest Hill North & West
High Park
Westmount
Wexford
Willowdale
North Toronto West
The Annex
Parkdale
Canada Post Gateway P

AttributeError: 'NoneType' object has no attribute 'latitude'

#### TRIED FINDING THE LATITUDE AND LONGITUDE OF THE POSTAL CODE (USING GEOCODER)

In [92]:
# import library
import geocoder

# Create new columns
Toronto_data['Latitude'] = None
Toronto_data['Longitude'] = None

# initiatize variable to None
coordinates = None

# iterate over every row to find its latitude and logitude
for index, row in Toronto_data.iterrows():
    Postal_code = row['Postal code']
    # While loop till we get locations
    while (coordinates == None):
        g = geocoder.google('{}, Toronto, Ontario'.format(Postal_code))
        coordinates = g.latlng
    
    latitude = coordinates[0]
    longitude = coordinates[1]
    
    # Add it to the new columns
    row['Latitude'] = latitude
    row['Longitude'] = longitude

KeyError: 'Postal code'

#### USING THE GIVEN COORDINATES FILE 

In [106]:
# Read the csv file containing the coordinates
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [108]:
coordinates_df.shape

(103, 3)

In [109]:
# Adding the Latitude and Longitude values 
Toronto_data['Latitude'] = coordinates_df['Latitude']
Toronto_data['Longitude'] = coordinates_df['Longitude']
Toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### END OF WEEK 3 ASSIGNMENT